## Ensure a Minikube cluster is running

- If you created a Minikube cluster with the command `minikube start --nodes 3` and it is still running:
  - Do nothing
- If you have a stopped (not deleted) Minikube cluster that was created with the command `minikube start --nodes 3`:
  - Run the command `minikube start`
- If you don't have a Minikube cluster (or deleted your previous one):
  - Run the command `minikube start --nodes 3`  

## Build custom Docker images on the host machine

- Builds a Docker image `hello-app:1.0` from the subfolder `v1`
  - The Dockerfile uses `nginx:latest` as the base image, and sets the default web page to `Hello (version 1)`.
- Builds a Docker image `hello-app:2.0` from the subfolder `v2`
  - The Dockerfile uses `nginx:latest` as the base image, and sets the default web page to `Hello (version 2)`.

In [1]:
!docker build -t hello-app:1.0 -f v1/Dockerfile v1/.
!docker build -t hello-app:2.0 -f v2/Dockerfile v2/.
!docker image ls | grep hello-app

failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending build context to Docker daemon  3.072kB
Step 1/3 : FROM nginx:latest
 ---> a8758716bb6a
Step 2/3 : WORKDIR /usr/share/nginx/html
 ---> Running in a9038fe0eb31
Removing intermediate container a9038fe0eb31
 ---> 472a5c4fdb53
Step 3/3 : COPY index.html index.html
 ---> 6bb0f5603201
Successfully built 6bb0f5603201
Successfully tagged hello-app:1.0
failed to fetch metadata: fork/exec /usr/local/lib/docker/cli-plugins/docker-buildx: no such file or directory

DEPRECATED: The legacy builder is deprecated and will be removed in a future release.
            Install the buildx component to build images with BuildKit:
            https://docs.docker.com/go/buildx/

Sending bu

## Load the Docker images from the host machine into the Minikube nodes.

In [2]:
!minikube image load hello-app:1.0
!minikube image load hello-app:2.0
!minikube image ls | grep hello-app

gcr.io/google-samples/hello-app:2.0
gcr.io/google-samples/hello-app:1.0
docker.io/library/hello-app:2.0
docker.io/library/hello-app:1.0


## Create a Deployment (version 1)

- The Deployment definition is in the YAML file `manifests/hello-deployment.yaml`.

In [3]:
!kubectl create -f manifests/hello-deployment.yaml

deployment.apps/hello-dep created


## Let's look at the Deployment's YAML.

**Note:**

- The number of `replicas` is set to 3.
- The Pod template's `labels` are `app: hello-dep`
  - The Deployment's `matchLabels` match these labels
    - Therefore, the Deployment will create 3 replicas from the Pod template.
  - The Deployment uses a `RollingUpdate` `strategy` `type` with `maxSurge` and `maxUnavailable` set to 1.
    - Therefore, there can exist:
      - A maximum of 4 (`replicas` + `maxSurege`) Pod instances during the update.
      - A minimum of 2 (`replicas` - `maxUnavailable`) Pod instances during the update.
- The Deployment uses a `revisionHistoryLimit` of 3.
  - Therefore, a maximum of 3 ReplicaSet versions will be kept in the revistion history.
- The Pod template's containers are:
  - Based on the Docker `hello-app:1.0` image (version 1).
  - Listening on `containerPort` 8080.

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-dep                # the Deployment's name
  namespace: default
spec:
  replicas: 3                    # the Deployment defines three Pod replicas
  revisionHistoryLimit: 3        # the Deployment defines a maximum number of ReplicaSet revisions in history to 3
  strategy:
    type: RollingUpdate          # the Deployment uses a RollingUpdate strategy
    rollingUpdate:
      maxSurge: 1                # a maximum of 3+1 Pod instances can exist during the update
      maxUnavailable: 1          # a minimum of 3-1 Pod instances can exist during the update
  selector:
    matchLabels:
      app: hello-dep             # the Deployment's matchLabels match the Pod template's labels below
  template:
    metadata:
      labels:
        app: hello-dep           # the Pod template defines one label (app: hello-dep)
    spec:
      containers:
      - image: hello-app:1.0     # the Pod template's containers are based on the hello-app:1.0 image (version 1)
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi      
        imagePullPolicy: Never
        name: hello-dep          # the container's name
        ports:
        - containerPort: 8080    # the containers are listening on port 8080
```

In [4]:
!cat manifests/hello-deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-dep
  namespace: default
spec:
  replicas: 3
  revisionHistoryLimit: 3
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
  selector:
    matchLabels:
      app: hello-dep
  template:
    metadata:
      labels:
        app: hello-dep
    spec:
      containers:
      - image: hello-app:1.0 # change this to: hello-app:2.0
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi      
        imagePullPolicy: Never
        name: hello-dep
        ports:
        - containerPort: 8080

## Get Deployment rollout status

- Note that the Deployment was rolled out successfully.

In [5]:
!kubectl rollout status deployment/hello-dep

deployment "hello-dep" successfully rolled out


## List Pods

- We see that 3 Pod replicas are running.

In [6]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                         READY   STATUS    RESTARTS   AGE   IP           NODE           NOMINATED NODE   READINESS GATES
hello-dep-59dffcc68d-rnz5h   1/1     Running   0          27s   10.244.2.3   minikube-m03   <none>           <none>
hello-dep-59dffcc68d-smkks   1/1     Running   0          27s   10.244.1.3   minikube-m02   <none>           <none>
hello-dep-59dffcc68d-x9zzn   1/1     Running   0          27s   10.244.0.5   minikube       <none>           <none>


## Get the default web page in one of the pods

- Use one of the pod names (above) to execute the command `-- curl http://localhost` in its container.
- We see that the default web page returns `Hello (version 1)` as defined in image `hello-app:1.0`.

In [7]:
POD_NAME = !kubectl get pods -o jsonpath='{.items[0].metadata.name}'
POD_NAME = POD_NAME[0]
!kubectl exec {POD_NAME} -- curl http://localhost

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0  36796      0 --:--:-- --:--:-- --:--:-- 17000
Hello (version 1)

## List ReplicaSets

- We see that the ReplicaSet contains 3 Pod replicas with containers based on image `hello-app:1.0`.

In [8]:
#!kubectl get rs -o wide
!kubectl get replicasets -o wide

NAME                   DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES          SELECTOR
hello-dep-59dffcc68d   3         3         3       39s   hello-dep    hello-app:1.0   app=hello-dep,pod-template-hash=59dffcc68d


## Create a Deployment (version 2)

- Open the YAML file `manifests/hello-deployment.yaml`.
- Change the image version from `1.0` to `2.0`.
- Save the file.

```bash
apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-dep
  namespace: default
spec:
  replicas: 3
  revisionHistoryLimit: 3
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
  selector:
    matchLabels:
      app: hello-dep
  template:
    metadata:
      labels:
        app: hello-dep
    spec:
      containers:
      - image: hello-app:2.0 # change this to: hello-app:2.0
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi      
        imagePullPolicy: Never
        name: hello-dep
        ports:
        - containerPort: 8080
```

In [9]:
!cat manifests/hello-deployment.yaml

apiVersion: apps/v1
kind: Deployment
metadata:
  name: hello-dep
  namespace: default
spec:
  replicas: 3
  revisionHistoryLimit: 3
  strategy:
    type: RollingUpdate
    rollingUpdate:
      maxSurge: 1
      maxUnavailable: 1
  selector:
    matchLabels:
      app: hello-dep
  template:
    metadata:
      labels:
        app: hello-dep
    spec:
      containers:
      - image: hello-app:2.0 # change this to: hello-app:2.0
        resources:
          requests:
            cpu: 100m
            memory: 128Mi
          limits:
            cpu: 250m
            memory: 256Mi      
        imagePullPolicy: Never
        name: hello-dep
        ports:
        - containerPort: 8080

## Deploy the updated Deployment's YAML file

In [10]:
!kubectl apply -f manifests/hello-deployment.yaml

deployment.apps/hello-dep configured


## Get Deployment rollout status

- We see that the updated Deployment's YAML definition was rolled out successfully.

In [11]:
!kubectl rollout status deployment/hello-dep

deployment "hello-dep" successfully rolled out


## List Pods

- We see that three Pods are running.

In [12]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                         READY   STATUS    RESTARTS   AGE   IP           NODE           NOMINATED NODE   READINESS GATES
hello-dep-64bb85f9cc-b8gwm   1/1     Running   0          14s   10.244.1.4   minikube-m02   <none>           <none>
hello-dep-64bb85f9cc-fj2nj   1/1     Running   0          15s   10.244.2.4   minikube-m03   <none>           <none>
hello-dep-64bb85f9cc-zsxsr   1/1     Running   0          15s   10.244.0.6   minikube       <none>           <none>


## Get the default web page in one of the pods

- Use one of the pod names (above) to execute the command `-- curl http://localhost` in its container.
- We see that the default web page returns `Hello (version 2)` as defined in image `hello-app:2.0`.

In [13]:
POD_NAME = !kubectl get pods -o jsonpath='{.items[0].metadata.name}'
POD_NAME = POD_NAME[0]
!kubectl exec {POD_NAME} -- curl http://localhost

Hello (version 2)  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0  48433      0 --:--:-- --:--:-- --:--:-- 17000


## List ReplicaSets

- Notice that:
  - The current ReplicaSet contains 3 Pod replicas with containers based on image `hello-app:2.0`.
  - The previous ReplicaSet is kept in the revision history with `DESIRED`, `CURRENT` and `READY` set to 0 (no Pods running).
    - This is because `revisionHistoryLimit` is set to 3 in the Deployment's YAML file.

In [14]:
#!kubectl get rs -o wide
!kubectl get replicasets -o wide

NAME                   DESIRED   CURRENT   READY   AGE   CONTAINERS   IMAGES          SELECTOR
hello-dep-59dffcc68d   0         0         0       92s   hello-dep    hello-app:1.0   app=hello-dep,pod-template-hash=59dffcc68d
hello-dep-64bb85f9cc   3         3         3       30s   hello-dep    hello-app:2.0   app=hello-dep,pod-template-hash=64bb85f9cc


## Rollback the Deployment

- Here we are using `--to-revision 1` to roll back the Deployment to version 1.

In [15]:
#!kubectl rollout undo deployment/hello-dep
!kubectl rollout undo deployment/hello-dep --to-revision 1

deployment.apps/hello-dep rolled back


## Get Deployment rollout status

- We see that the rolled back Deployment was rolled out successfully.

In [16]:
!kubectl rollout status deployment/hello-dep

deployment "hello-dep" successfully rolled out


## List Pods

- We see that 3 Pods are running.

In [17]:
#!kubectl get po -o wide
!kubectl get pods -o wide

NAME                         READY   STATUS    RESTARTS   AGE   IP           NODE           NOMINATED NODE   READINESS GATES
hello-dep-59dffcc68d-6vzq8   1/1     Running   0          14s   10.244.1.5   minikube-m02   <none>           <none>
hello-dep-59dffcc68d-rr6q2   1/1     Running   0          13s   10.244.0.7   minikube       <none>           <none>
hello-dep-59dffcc68d-zbg6p   1/1     Running   0          14s   10.244.2.5   minikube-m03   <none>           <none>


## Get the default web page in one of the pods

- Use one of the pod names (above) to execute the command `-- curl http://localhost` in its container.
- We see that the default web page returns `Hello (version 1)` as defined in image `hello-app:1.0`.

In [18]:
POD_NAME = !kubectl get pods -o jsonpath='{.items[0].metadata.name}'
POD_NAME = POD_NAME[0]
!kubectl exec {POD_NAME} -- curl http://localhost

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100    17  100    17    0     0  41262      0 --:--:-- --:--:-- --:--:-- 17000Hello (version 1)


## List ReplicaSets

- We see that the ReplicaSet contains 3 Pod replicas with containers based on image `hello-app:1.0`.

In [19]:
#!kubectl get rs -o wide
!kubectl get replicasets -o wide

NAME                   DESIRED   CURRENT   READY   AGE     CONTAINERS   IMAGES          SELECTOR
hello-dep-59dffcc68d   3         3         3       2m11s   hello-dep    hello-app:1.0   app=hello-dep,pod-template-hash=59dffcc68d
hello-dep-64bb85f9cc   0         0         0       69s     hello-dep    hello-app:2.0   app=hello-dep,pod-template-hash=64bb85f9cc


## Delete the Deployment

In [20]:
!kubectl delete -f manifests/hello-deployment.yaml

deployment.apps "hello-dep" deleted


## List Deployments, ReplicaSets and Pods

- We see that deleting the Deployment also deleted its associated ReplicaSet and Pods.

In [21]:
#!kubectl get deploy
#!kubectl get rs
#!kubectl get po
!kubectl get deployments
!kubectl get replicasets
!kubectl get pods

No resources found in default namespace.
No resources found in default namespace.
No resources found in default namespace.


## Remove the Docker images from the Minikube nodes

In [22]:
!minikube image rm hello-app:1.0
!minikube image rm hello-app:2.0
!minikube image ls | grep hello-app

gcr.io/google-samples/hello-app:2.0
gcr.io/google-samples/hello-app:1.0


## Delete the custom Docker images from the host machine

In [23]:
!docker rmi hello-app:1.0
!docker rmi hello-app:2.0
!docker image ls | grep hello-app

Untagged: hello-app:1.0
Deleted: sha256:6bb0f5603201854bbba4e8f7fb121ecf2c275ccb538dcd26313981bae8a218f0
Deleted: sha256:4772c4ca6d3534ac11bd62de02a330dc68e1a94245dc7105b524e574b0b3e05f
Untagged: hello-app:2.0
Deleted: sha256:dedea9c40e981b4ce1a5e02f87eeda24fb4a377a58c07e37701dbb492b1be24c
Deleted: sha256:f4f7496c3a2f79756d76413953aecbe8b909be493aacc5c7bd0b5d5d66f3e1e3
Deleted: sha256:472a5c4fdb53522ba0a83f110f4a842e1a937da705171fd3918ae2acbc226572
